In [1]:
import midi
import numpy as np

In [6]:
pattern = midi.read_midifile('scale2.mid')
print pattern

midi.Pattern(format=1, resolution=960, tracks=\
[midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[7, 161, 32]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[0]),
   midi.TrackNameEvent(tick=0, text='Untitled\x00', data=[85, 110, 116, 105, 116, 108, 101, 100, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.NoteOnEvent(tick=0, channel=0, data=[72, 80]),
   midi.NoteOffEvent(tick=960, channel=0, data=[72, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[74, 80]),
   midi.NoteOffEvent(tick=960, cha

In [5]:
midi.write_midifile("example.mid", pattern)

In [7]:
pattern3 = midi.read_midifile('tempo.mid')
print pattern3

midi.Pattern(format=1, resolution=960, tracks=\
[midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[4, 147, 224]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[3, 2, 24, 8]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[0]),
   midi.TrackNameEvent(tick=0, text='Untitled\x00', data=[85, 110, 116, 105, 116, 108, 101, 100, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.NoteOnEvent(tick=0, channel=0, data=[72, 80]),
   midi.NoteOffEvent(tick=960, channel=0, data=[72, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[74, 80]),
   midi.NoteOffEvent(tick=960, ch

In [3]:
pattern3 = midi.read_midifile('tempo.mid')
print pattern3

midi.Pattern(format=1, resolution=960, tracks=\
[midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[4, 147, 224]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[8]),
   midi.TrackNameEvent(tick=0, text='Untitled\x00', data=[85, 110, 116, 105, 116, 108, 101, 100, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.NoteOnEvent(tick=0, channel=0, data=[69, 80]),
   midi.NoteOnEvent(tick=0, channel=0, data=[71, 80]),
   midi.NoteOnEvent(tick=240, channel=0, data=[73, 80]),
   midi.NoteOffEvent(tick=240, ch

In [4]:
pattern3 = midi.read_midifile('simple.mid')
print pattern3

midi.Pattern(format=1, resolution=960, tracks=\
[midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[4, 147, 224]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[8]),
   midi.TrackNameEvent(tick=0, text='Untitled\x00', data=[85, 110, 116, 105, 116, 108, 101, 100, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.NoteOnEvent(tick=0, channel=0, data=[71, 80]),
   midi.NoteOnEvent(tick=0, channel=0, data=[69, 80]),
   midi.NoteOffEvent(tick=960, channel=0, data=[71, 0]),
   midi.NoteOnEvent(tick=1920, ch

In [81]:
class MidiContainer:
    '''
    The MidiContainer class contains a vector representation of 
    the midi track to be fed to our neural net. It is a 67 x n matrix,
    contents specified in the README. 
    '''
    def __init__(self):
        self.data = np.array([], dtype=np.int64).reshape(7,0) #todo change this to 3+16*4
        self.curr_bpm = 120
        self.curr_instrument = 0
        self.abs_time = 0
        self.active_pitches = {} #[pitch] = (velocity, time note was turned on)
        
    def note_to_vector(self, pitch, time):
        velocity = self.active_pitches[pitch][0]
        note_start = self.active_pitches[pitch][1]
        return np.vstack((self.curr_instrument, pitch, velocity, time + self.abs_time - note_start))
    
    def add_data(self, note_vector, note_start):
        print 'for note ', note_vector , ' and start ', note_start , ' and abs time ', self.abs_time
        event_column = np.vstack((note_start, self.curr_bpm, note_vector,self.abs_time))
        self.data = np.hstack((self.data, event_column))
    

In [82]:
def midi_to_vector(fname):
    '''
    Given a filename of a midi to read, returns a MidiContainer object of its vector representation.
    '''
    pattern = midi.read_midifile(fname)
    midi_vector = MidiContainer()
    for track in pattern:
        print 'start track!'
        for event in track:
            print 'event: ', event
            if isinstance(event, midi.SetTempoEvent): 
                midi_vector.curr_bpm = event.get_bpm()
            if isinstance(event, midi.EndOfTrackEvent): #first metadata track
                continue 
            if isinstance(event, midi.ProgramChangeEvent):
                midi_vector.curr_instrument = event.get_value()
            if isinstance(event, midi.NoteOnEvent):
                midi_vector.abs_time += event.tick
                midi_vector.active_pitches[event.get_pitch()] = (event.get_velocity(), midi_vector.abs_time)
                print 'on abs time now ', midi_vector.abs_time
                print 'on pitches is ', midi_vector.active_pitches
            if isinstance(event, midi.NoteOffEvent):
                pitch = event.get_pitch()
                time = event.tick
                print 'time is ', time 
                if pitch in midi_vector.active_pitches:
                    note_vec = midi_vector.note_to_vector(pitch, time)
                    midi_vector.add_data(note_vec, midi_vector.active_pitches[pitch][1]) #ugh bad data abstraction
                    midi_vector.active_pitches.pop(pitch, None)
                    midi_vector.abs_time += event.tick
                    print 'off abs time now ', midi_vector.abs_time
                
            
    return midi_vector

In [83]:
a = midi_to_vector('simple.mid')
print a.data

start track!
event:  midi.SetTempoEvent(tick=0, data=[4, 147, 224])
event:  midi.KeySignatureEvent(tick=0, data=[0, 0])
event:  midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8])
event:  midi.EndOfTrackEvent(tick=0, data=[])
start track!
event:  midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0])
event:  midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0])
event:  midi.ProgramChangeEvent(tick=0, channel=0, data=[8])
event:  midi.TrackNameEvent(tick=0, text='Untitled\x00', data=[85, 110, 116, 105, 116, 108, 101, 100, 0])
event:  midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127])
event:  midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0])
event:  midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0])
event:  midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127])
event:  midi.NoteOnEvent(tick=0, channel=0, data=[71, 80])
on abs time now  0
on pitches is  {71: (80, 0)}
event:  midi.NoteOnEvent(tick=0, channel=0, data=[69, 80])
on abs time now  0
on pitches is  {69:

In [65]:
pattern3 = midi.read_midifile('simple.mid')
print pattern3

midi.Pattern(format=1, resolution=960, tracks=\
[midi.Track(\
  [midi.SetTempoEvent(tick=0, data=[4, 147, 224]),
   midi.KeySignatureEvent(tick=0, data=[0, 0]),
   midi.TimeSignatureEvent(tick=0, data=[4, 2, 24, 8]),
   midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
  [midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[8]),
   midi.TrackNameEvent(tick=0, text='Untitled\x00', data=[85, 110, 116, 105, 116, 108, 101, 100, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[0, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[32, 0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 127]),
   midi.NoteOnEvent(tick=0, channel=0, data=[71, 80]),
   midi.NoteOnEvent(tick=0, channel=0, data=[69, 80]),
   midi.NoteOffEvent(tick=960, channel=0, data=[71, 0]),
   midi.NoteOnEvent(tick=1920, ch